# 3. Cassandra to Arrow

We use some code from the Cassandra server to read the SSTable, but instead of de/serializing to/from CQL, we use an [Arrow IPC stream](http://arrow.apache.org/), which is stored in a columnar format and better suited for analytics.

Data transformations:

1. SSTable on disk
2. Deserialized into Java Object in C* server
3. Client makes request to server (not to C* DB)
4. Data serialized via Arrow IPC stream
5. Sent across network
6. Arrow IPC stream received by client
7. Transformed into Arrow Table / cuDF

**Pros:**
- doesn't make request to the main Cassandra DB, which lessens the load and allows for other operations to run
- less de/serialization involved using the Arrow IPC stream

**Cons:**
- don't want to have to start Cassandra or use the JVM
- complex architecture

In [1]:
import pyarrow as pa
# my local computer doesn't have cudf installed
# import cudf
import socket

HOST = '127.0.0.1'
PORT = 9143

In [2]:
# read data from socket
def fetch_data():
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.connect((HOST, PORT))
        s.sendall(b'hello world\n')
        data = b''
        while True:
            newdata = s.recv(1024)
            if not newdata:
                break
            data += newdata
    return data

In [6]:
buffer = fetch_data()
reader = pa.ipc.open_stream(buffer)
arrow_table = reader.read_all()
arrow_table.to_pandas() # for visualization

,_timestamp,partition key,clustering key,data,sensor_value,station_id
0,1924-04-12 00:00:00.002,{'org.apache.cassandra.db.marshal.UUIDType': b...,1970-01-01 00:00:00.002,"ue sapien et, fermentum neque. Pellentesque mo...",95.759791,b'(\xdfc\xb7\xccWC\xcb\x97R\xfa\xe6\x9d\x16S\xda'
1,1924-04-12 00:00:00.009,{'org.apache.cassandra.db.marshal.UUIDType': b...,1970-01-01 00:00:00.009,ctus mauris nec urna. Duis sit amet enim trist...,106.497951,b'(\xdfc\xb7\xccWC\xcb\x97R\xfa\xe6\x9d\x16S\xda'
2,1924-04-12 00:00:00.002,{'org.apache.cassandra.db.marshal.UUIDType': b...,1970-01-01 00:00:00.002,ere purus vitae vehicula porttitor. Cum sociis...,100.090271,b'(\xdfc\xb7\xccWC\xcb\x97R\xfa\xe6\x9d\x16S\xda'
3,1924-04-12 00:00:00.001,{'org.apache.cassandra.db.marshal.UUIDType': b...,1970-01-01 00:00:00.001,"luctus est, et vulputate odio felis interdum m...",94.181632,b'(\xdfc\xb7\xccWC\xcb\x97R\xfa\xe6\x9d\x16S\xda'
4,1924-04-12 00:00:00.000,{'org.apache.cassandra.db.marshal.UUIDType': b...,1970-01-01 00:00:00.000,imperdiet sed est. Aliquam a quam condimentum...,99.909094,b'(\xdfc\xb7\xccWC\xcb\x97R\xfa\xe6\x9d\x16S\xda'
...,...,...,...,...,...,...
995,1924-04-12 00:00:00.003,{'org.apache.cassandra.db.marshal.UUIDType': b...,1970-01-01 00:00:00.003,ulum vehicula euismod purus ac tempus. Nam mol...,105.890743,b'(\xdfc\xb7\xccWC\xcb\x97R\xfa\xe6\x9d\x16S\xda'
996,1924-04-12 00:00:00.006,{'org.apache.cassandra.db.marshal.UUIDType': b...,1970-01-01 00:00:00.006,rat nibh. Vivamus eu quam id mi gravida porta ...,103.857088,b'(\xdfc\xb7\xccWC\xcb\x97R\xfa\xe6\x9d\x16S\xda'
997,1924-04-12 00:00:00.000,{'org.apache.cassandra.db.marshal.UUIDType': b...,1970-01-01 00:00:00.000,pien.\nDonec sit amet est eu libero malesuada ...,95.420393,b'(\xdfc\xb7\xccWC\xcb\x97R\xfa\xe6\x9d\x16S\xda'
998,1924-04-12 00:00:00.001,{'org.apache.cassandra.db.marshal.UUIDType': b...,1970-01-01 00:00:00.001,"sem, gravida vitae commodo a, vestibulum plac...",103.726586,b'(\xdfc\xb7\xccWC\xcb\x97R\xfa\xe6\x9d\x16S\xda'
